In this example, we use CellCnn to analyze a mass cytometry dataset acquired to characterize human natural killer (NK) cell diversity and associate NK cell subsets with genetic and environmental factors, namely prior Cytomegalovirus (CMV) infection [1]. This dataset comprises mass cytometry measurements of 36 markers, including 28 NK cell receptors, for PBMC samples of 20 donors with varying serology for CMV. 

We will train CellCnn to identify CMV seropositivity-associated cell populations within the **manually gated NK cell compartment**. To run this example, please download the [NK cell dataset](http://www.imsb.ethz.ch/research/claassen/Software/cellcnn.html) and place the decompressed folder in the cellCnn/examples directory.

[1] Horowitz, A. et al. Genetic and environmental determinants of human NK cell diversity revealed by mass cytometry. Sci. Transl. Med. 5 (2013).


In [1]:
import os, sys, errno, glob, fcm
import numpy as np
import pandas as pd

import cellCnn
from cellCnn.utils import ftrans, mkdir_p, get_items
from cellCnn.model import CellCnn
from cellCnn.plotting import plot_results
from sklearn.metrics import roc_auc_score

%pylab inline


Using Theano backend.


Populating the interactive namespace from numpy and matplotlib


In [2]:
# define input and output directories
WDIR = os.path.join(cellCnn.__path__[0], 'examples')
FCS_DATA_PATH = os.path.join(WDIR, 'NK_cell_dataset', 'gated_NK')

# define output directory
OUTDIR = os.path.join(WDIR, 'output_NK')
mkdir_p(OUTDIR)

In [25]:
# look at the measured markers
data_fcs = fcm.loadFCS(glob.glob(FCS_DATA_PATH + '/*.fcs')[0], transform=None, auto_comp=False)
print data_fcs.channels


['Time', 'Cell_length', 'CD3', 'Dead', '(La139)Dd', 'CD27', 'CD19', 'CD4', 'CD8', 'CD57', '2DL1-S1', 'TRAIL', '2DL2-L3-S2', 'CD16', 'CD10', '3DL1-S1', 'CD117', '2DS4', 'ILT2-CD85j', 'NKp46', 'NKG2D', 'NKG2C', '2B4', 'CD33', 'CD11b', 'NKp30', 'CD122', '3DL1', 'NKp44', 'CD127', '2DL1', 'CD94', 'CD34', 'CCR7', '2DL3', 'NKG2A', 'HLA-DR', '2DL4', 'CD56', '2DL5', 'CD25', 'DNA1', 'DNA2']


In [4]:
# select the relevant markers for further analysis
markers = ['CD3', 'CD27', 'CD19', 'CD4', 'CD8', 'CD57', '2DL1-S1', 'TRAIL', '2DL2-L3-S2',
           'CD16', 'CD10', '3DL1-S1', 'CD117', '2DS4', 'ILT2-CD85j', 'NKp46', 'NKG2D',
           'NKG2C', '2B4', 'CD33', 'CD11b', 'NKp30', 'CD122', '3DL1', 'NKp44', 'CD127', '2DL1',
           'CD94', 'CD34', 'CCR7', '2DL3', 'NKG2A', 'HLA-DR', '2DL4', 'CD56', '2DL5', 'CD25']
marker_idx = [data_fcs.channels.index(label) for label in markers]
nmark = len(markers)

In [5]:
# load the sample names and corresponding labels (0: CMV-, 1: CMV+), here from a CSV file
# prior CMV infection status is obtained from the original study (Horowitz et al. 2013)
csv_file = 'NK_fcs_samples_with_labels.csv'
fcs_info = np.array(pd.read_csv(csv_file, sep=','))
sample_ids = fcs_info[:, 0]
sample_labels = fcs_info[:, 1].astype(int)

In [6]:
# set random seed for reproducible results
np.random.seed(12345)

# cofactor for arcsinh transformation
cofactor = 5

# split the fcs files into training, validation and test set
group1 = np.where(sample_labels == 0)[0]
group2 = np.where(sample_labels == 1)[0]
l1, l2 = len(group1), len(group2)
ntrain_per_class = 7
ntest_group1 = l1 - ntrain_per_class
ntest_group2 = l2 - ntrain_per_class

# get the sample indices
train_idx1 = list(np.random.choice(group1, size=ntrain_per_class, replace=False))
test_idx1 = [i for i in group1 if i not in train_idx1]
train_idx2 = list(np.random.choice(group2, size=ntrain_per_class, replace=False))
test_idx2 = [i for i in group2 if i not in train_idx2]

# load the training samples
group1_list, group2_list = [], []
for idx in train_idx1:
    fname = os.path.join(FCS_DATA_PATH, sample_ids[idx])
    x_full = np.asarray(fcm.loadFCS(fname, transform=None, auto_comp=False))
    x = ftrans(x_full[:,marker_idx], cofactor)
    group1_list.append(x)

for idx in train_idx2:
    fname = os.path.join(FCS_DATA_PATH, sample_ids[idx])
    x_full = np.asarray(fcm.loadFCS(fname, transform=None, auto_comp=False))
    x = ftrans(x_full[:,marker_idx], cofactor)
    group2_list.append(x)

# load the test samples
t_group1_list, t_group2_list = [], []
test_phenotypes = []
for idx in test_idx1:
    fname = os.path.join(FCS_DATA_PATH, sample_ids[idx])
    x_full = np.asarray(fcm.loadFCS(fname, transform=None, auto_comp=False))
    x = ftrans(x_full[:,marker_idx], cofactor)
    t_group1_list.append(x)
    test_phenotypes.append(0)

for idx in test_idx2:
    fname = os.path.join(FCS_DATA_PATH, sample_ids[idx])
    x_full = np.asarray(fcm.loadFCS(fname, transform=None, auto_comp=False))
    x = ftrans(x_full[:,marker_idx], cofactor)
    t_group2_list.append(x)
    test_phenotypes.append(1)

# finally prepare training and vallidation data
cut = int(.8 * len(group1_list))
train_samples = group1_list[:cut] + group2_list[:cut]
train_phenotypes = [0] * len(group1_list[:cut]) + [1] * len(group2_list[:cut])
valid_samples = group1_list[cut:] + group2_list[cut:]
valid_phenotypes = [0] * len(group1_list[cut:]) + [1] * len(group2_list[cut:])
test_samples = t_group1_list + t_group2_list


In [7]:
# run a CellCnn analysis
model = CellCnn(ncell=200, nsubset=1000, verbose=0)

model.fit(train_samples=train_samples, train_phenotypes=train_phenotypes,
          valid_samples=valid_samples, valid_phenotypes=valid_phenotypes, outdir=OUTDIR)


Generating multi-cell inputs...
Done.
Number of filters: 7
Cells pooled: 1
2000/2000 [==============================] - 0s     

In [24]:
# now make predictions using the trained model
train_pred = model.predict(train_samples)
valid_pred = model.predict(valid_samples)
test_pred = model.predict(test_samples)
print train_pred, train_phenotypes
print valid_pred, valid_phenotypes
print test_pred, test_phenotypes

# calculate area under the ROC curve
train_auc = roc_auc_score(train_phenotypes, train_pred[:,1])
valid_auc = roc_auc_score(valid_phenotypes, valid_pred[:,1])
test_auc = roc_auc_score(test_phenotypes, test_pred[:,1])
print train_auc, valid_auc, test_auc

Predictions based on multi-cell inputs containing 6320 cells.
Predictions based on multi-cell inputs containing 3790 cells.
Predictions based on multi-cell inputs containing 5652 cells.
[[ 0.82009562  0.17990439]
 [ 0.90225913  0.09774087]
 [ 0.90180832  0.09819169]
 [ 0.9617671   0.0382329 ]
 [ 0.94296861  0.05703137]
 [ 0.19711297  0.80288704]
 [ 0.14592338  0.8540766 ]
 [ 0.0108008   0.9891992 ]
 [ 0.04772658  0.95227343]
 [ 0.02897164  0.97102835]] [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[[ 0.94501744  0.05498259]
 [ 0.67689306  0.32310692]
 [ 0.039119    0.96088101]
 [ 0.33426374  0.66573628]] [0, 0, 1, 1]
[[ 0.89207431  0.1079257 ]
 [ 0.91889727  0.08110273]
 [ 0.37125671  0.62874329]
 [ 0.21356793  0.78643207]
 [ 0.00876355  0.99123645]
 [ 0.1790728   0.82092718]] [0, 0, 0, 0, 1, 1]
1.0 1.0 1.0


In [23]:
# plot the results of the CellCnn analysis for the test samples in the output directory
plot_results(model.results, test_samples, test_phenotypes,
             markers, OUTDIR, filter_response_thres=0,
             filter_diff_thres=0.2, group_a='CMV-', group_b='CMV+')

Loading the weights of consensus filters.


[(0, 0)]

In [18]:
# The following plots are generated in the output directory `OUTDIR`.

# In this example, we have two selected filters: filter 0 and filter 1.
# Filter 0 has high weights for the markers CD16, NKG2C and CD94
# and it is positively associated with class 1, that is CMV+ samples.

from IPython.display import IFrame
IFrame("./output_NK/consensus_filter_weights.pdf", width=600, height=350)

In [17]:
# We also see that filter 0 is more discriminative than filter 1,
# because the average cell filter response difference between CMV+
# and CMV- validation samples is higher for filter 0.

# The `filter_diff_thres` parameter (that is given as input to the `plot_results` function)
# is a threshold that defines which filters should be kept for further analysis.
# Given an array `filter_diff` of average cell filter response
# differences between classes (y-axis on the plot), sorted in decreasing order,
# we keep a filter `i, i > 0` if it holds that
# `filter_diff[i-1] - filter_diff[i] < filter_diff_thres * filter_diff[i-1]`.
# The default value is `filter_diff_thres`=0.2

IFrame("./output_NK/filter_response_differences.pdf", width=600, height=350)

In [12]:
# For filters selected as discriminative via the previously described step,
# we can inspect the cumulative distribution function (CDF) of the cell filter response.
# Based on this, we can pick a threshold on the cell filter response of the selected
# cell population. The default value for this threshold is `filter_response_thres`=0.

IFrame("./output_NK/cdf_filter_0.pdf", width=600, height=350)

In [13]:
# Marker abundance histograms of each selected cell population are overlaid
# with the corresponding marker abundance histograms of all cells. The distance between
# distributions is quantified via the Kolmogorov-Smirnov (KS) statistic.

IFrame("./output_NK/selected_population_distribution_filter_0.pdf", width=600, height=350)

In [20]:
# We plot again, now using a more stringent cutoff on the cell filter response of
# the selected cell population.

plot_results(model.results, test_samples, test_phenotypes,
             markers, OUTDIR, filter_response_thres=0.7,
             filter_diff_thres=0.2, group_a='CMV-', group_b='CMV+')

IFrame("./output_NK/cdf_filter_0.pdf", width=600, height=350)

Loading the weights of consensus filters.


In [21]:
# The selected cell population is now a CD57+ NKG2C+ NK cell subset

IFrame("./output_NK/selected_population_distribution_filter_0.pdf", width=600, height=350)

In [22]:
# For binary classification problems, we also get a boxplot of frequencies of the selected
# cell population in samples of the two classes.

IFrame("./output_NK/selected_population_boxplot_filter_0.pdf", width=600, height=350)